In [1]:
import pandas
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates
from matplotlib import rcParams
rcParams.update({'figure.autolayout': True})
pandas.set_option('display.max_columns', 10)
pandas.set_option('display.width', 350)


**I download the xlsx and put it under the same file with my code**

In [3]:
data = pandas.read_excel("Adops & Data Scientist Sample Data.xlsx",index_col=0)
print(data.shape)

data.head()


(3553, 3)


,user_id,country_id,site_id
ts,,,
2019-02-01 00:01:24,LC36FC,TL6,N0OTG
2019-02-01 00:10:19,LC39B6,TL6,N0OTG
2019-02-01 00:21:50,LC3500,TL6,N0OTG
2019-02-01 00:22:50,LC374F,TL6,N0OTG
2019-02-01 00:23:44,LCC1C3,TL6,QGO3G


**For instance, in row 2 you will see the following row:**
- "2019-02-01 00:01:24	LC36FC	TL6	N0OTG"
- This tells us that at "2019-02-01 00:01:24" user "LC36FC" from country "TL6" visited website "N0OTG".

## Q1 Consider only the rows with country_id = "BDV" (there are 844 such rows). 
### - For each site_id, we can compute the number of unique user_id's found in these 844 rows. Which site_id has the largest number of unique users? And what's the number?

In [4]:
data1=data.query('country_id==\"BDV\"')
data1

,user_id,country_id,site_id
ts,,,
2019-02-01 00:47:58,LC3C22,BDV,N0OTG
2019-02-01 02:46:15,LC375A,BDV,5NPAU
2019-02-01 06:44:42,LC392E,BDV,N0OTG
2019-02-01 07:36:56,LC3FAE,BDV,5NPAU
2019-02-01 08:58:35,LC3AFF,BDV,3POLC
...,...,...,...
2019-02-07 22:18:59,LC3ECF,BDV,5NPAU
2019-02-07 22:26:28,LC3D63,BDV,5NPAU
2019-02-07 22:27:29,LC3005,BDV,5NPAU


In [5]:
data1.groupby('site_id')['user_id'].nunique()

site_id
3POLC      2
5NPAU    544
N0OTG     90
Name: user_id, dtype: int64

### answer:
- Thus the site with largest number of unique user number is 5NPAU
- the number is 544

## Q2 Between 2019-02-03 00:00:00 and 2019-02-04 23:59:59, there are four users who visited a certain site more than 10 times. 
### - Find these four users & which sites they (each) visited more than 10 times. (Simply provides four triples in the form (user_id, site_id, number of visits) in the box below.)

In [6]:
data=data.reset_index()
data['ts'] = pandas.to_datetime(data['ts'])
data.head()

,ts,user_id,country_id,site_id
0,2019-02-01 00:01:24,LC36FC,TL6,N0OTG
1,2019-02-01 00:10:19,LC39B6,TL6,N0OTG
2,2019-02-01 00:21:50,LC3500,TL6,N0OTG
3,2019-02-01 00:22:50,LC374F,TL6,N0OTG
4,2019-02-01 00:23:44,LCC1C3,TL6,QGO3G


In [7]:
data2=data
start_date = '2019-02-03 00:00:00'
end_date = '2019-02-04 23:59:59'
mask = (data2['ts'] > start_date) & (data2['ts'] <= end_date)
data2=data2.loc[mask]


In [8]:
df2 = data2.groupby(['user_id', 'site_id']).size().reset_index(name='count') 
df2.query('count>10')

,user_id,site_id,count
3,LC06C3,N0OTG,25
417,LC3A59,N0OTG,26
485,LC3C7E,3POLC,15
493,LC3C9D,N0OTG,17


## Anwser :
(LC06C3, N0OTG, 25), (LC3A59, N0OTG, 26), (LC3C7E, 3POLC, 15), (LC3C9D, N0OTG, 17)

## Q3 For each site, compute the unique number of users whose last visit (found in the original data set) was to that site. 
### -For instance, user "LC3561"'s last visit is to "N0OTG" based on timestamp data. Based on this measure, what are top three sites? (hint: site "3POLC" is ranked at 5th with 28 users whose last visit in the data set was to 3POLC; simply provide three pairs in the form (site_id, number of users).)

In [9]:
data3=data.groupby('user_id')

res = data3.apply(lambda x: x.sort_values(by='ts', ascending=False).head(1))
res=res.reset_index(drop=True)

In [10]:
#check 
res.query('user_id==\'LC3561\'')

,ts,user_id,country_id,site_id
581,2019-02-07 23:44:34,LC3561,TL6,N0OTG


In [11]:
result=res.groupby('site_id').count()
result.sort_values(by='user_id',ascending=False)

,ts,user_id,country_id
site_id,,,
5NPAU,992,992,992
N0OTG,561,561,561
QGO3G,289,289,289
GVOFK,42,42,42
3POLC,28,28,28
RT9Z6,2,2,2
EUZ/Q,1,1,1
JSUUP,1,1,1


### answer:
(5NPAU, 992), (N0OTG, 561), (QGO3G, 289)

## Q4 For each user, determine the first site he/she visited and the last site he/she visited based on the timestamp data. Compute the number of users whose first/last visits are to the same website. What is the number?

In [12]:
last=res
last=last.reset_index(drop=True)
last.drop('country_id', axis=1, inplace=True)
last=last.rename(columns={"ts": "last_ts", "site_id": "last_site_id"})
last.head()

,last_ts,user_id,last_site_id
0,2019-02-03 18:52:50,LC00C3,5NPAU
1,2019-02-04 11:35:10,LC01C3,5NPAU
2,2019-02-02 14:14:44,LC05C3,5NPAU
3,2019-02-07 01:16:12,LC06C3,N0OTG
4,2019-02-05 19:06:42,LC07C3,5NPAU


In [13]:
data4=data.groupby('user_id')

first = data4.apply(lambda x: x.sort_values(by='ts').head(1))
first=first.reset_index(drop=True)
first.drop('country_id', axis=1, inplace=True)
first=first.rename(columns={"ts": "first_ts", "site_id": "first_site_id"})
first.head()

,first_ts,user_id,first_site_id
0,2019-02-03 18:52:50,LC00C3,5NPAU
1,2019-02-04 11:35:10,LC01C3,5NPAU
2,2019-02-02 14:14:44,LC05C3,5NPAU
3,2019-02-01 22:49:39,LC06C3,N0OTG
4,2019-02-05 19:06:42,LC07C3,5NPAU


In [14]:
ans=pandas.merge(left=first, right=last, on='user_id')
ans.head()

,first_ts,user_id,first_site_id,last_ts,last_site_id
0,2019-02-03 18:52:50,LC00C3,5NPAU,2019-02-03 18:52:50,5NPAU
1,2019-02-04 11:35:10,LC01C3,5NPAU,2019-02-04 11:35:10,5NPAU
2,2019-02-02 14:14:44,LC05C3,5NPAU,2019-02-02 14:14:44,5NPAU
3,2019-02-01 22:49:39,LC06C3,N0OTG,2019-02-07 01:16:12,N0OTG
4,2019-02-05 19:06:42,LC07C3,5NPAU,2019-02-05 19:06:42,5NPAU


In [15]:
ans.query("first_site_id==last_site_id")

,first_ts,user_id,first_site_id,last_ts,last_site_id
0,2019-02-03 18:52:50,LC00C3,5NPAU,2019-02-03 18:52:50,5NPAU
1,2019-02-04 11:35:10,LC01C3,5NPAU,2019-02-04 11:35:10,5NPAU
2,2019-02-02 14:14:44,LC05C3,5NPAU,2019-02-02 14:14:44,5NPAU
3,2019-02-01 22:49:39,LC06C3,N0OTG,2019-02-07 01:16:12,N0OTG
4,2019-02-05 19:06:42,LC07C3,5NPAU,2019-02-05 19:06:42,5NPAU
...,...,...,...,...,...
1910,2019-02-02 13:58:18,LCFC38,5NPAU,2019-02-02 13:58:18,5NPAU
1911,2019-02-05 04:53:03,LCFC3B,N0OTG,2019-02-05 04:53:03,N0OTG
1912,2019-02-01 18:59:50,LCFC3D,N0OTG,2019-02-01 18:59:50,N0OTG
1913,2019-02-01 20:49:08,LCFC3E,5NPAU,2019-02-01 20:49:13,5NPAU


**If we conclude users who only visit once, the answer is 1670
However, this is meaningless, we only care about people who visit at least twice.**

In [16]:
ans.query("first_ts!= last_ts & first_site_id==last_site_id")

,first_ts,user_id,first_site_id,last_ts,last_site_id
3,2019-02-01 22:49:39,LC06C3,N0OTG,2019-02-07 01:16:12,N0OTG
10,2019-02-03 22:27:55,LC0C3E,N0OTG,2019-02-06 23:26:41,N0OTG
11,2019-02-06 11:37:50,LC0CC3,N0OTG,2019-02-07 11:51:14,N0OTG
14,2019-02-02 21:46:15,LC0FC3,N0OTG,2019-02-07 00:12:01,N0OTG
26,2019-02-01 09:12:18,LC22C3,3POLC,2019-02-01 18:51:23,3POLC
...,...,...,...,...,...
1905,2019-02-02 17:47:06,LCFBC3,5NPAU,2019-02-02 17:47:31,5NPAU
1908,2019-02-01 11:18:00,LCFC34,N0OTG,2019-02-02 23:30:56,N0OTG
1909,2019-02-02 01:43:41,LCFC36,5NPAU,2019-02-02 02:02:16,5NPAU
1913,2019-02-01 20:49:08,LCFC3E,5NPAU,2019-02-01 20:49:13,5NPAU


## Anwer:
**Then only 409 user_id are satisfied.**